In [60]:
import pandas as pd
import copy
# p.Count, p.Pos, p.CgRank, p.IsLiked, p.IsShared, p.IsProfileOpened, p.IsDownloaded, p.IsFollowed, p.Vplay98
df = pd.read_csv("cg_ranks_big.tsv", sep="\t", header=None, names=["cnt", "pos", "cg_rank", "is_liked", "is_shared", "is_profile_opened", "is_downloaded", "is_followed", "vplay98"])
df

,cnt,pos,cg_rank,is_liked,is_shared,is_profile_opened,is_downloaded,is_followed,vplay98
0,29594,7,creator-affinity-post-backPress,False,False,False,False,False,False
1,39,215,ffm-weighted-vplay98-tf_recall_multilingual_du...,False,False,True,False,False,True
2,3,21,engaged-creator-affinity-post,True,False,True,True,False,True
3,1,55,audio-id-trending-post,False,True,False,True,False,False
4,1,111,ffm-weighted-vplay98-tf_recall_multilingual_du...,True,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...
240742,1,83,audio-id-affinity-top-post,True,False,True,True,False,False
240743,9,20,lens-affinity-post,True,False,True,False,False,False
240744,18,310,ffm-vplay98-tf_recall_24hr_interaction,False,True,False,False,False,True
240745,116,39,sharecone-weighted-realtimecosine_150k-[1 1 1 ...,False,True,False,True,False,True


In [61]:
# unique cg_ranks
s = set(df["cg_rank"])
len(s), s

(28,
 {'audio-id-affinity-manual-post',
  'audio-id-affinity-top-post',
  'audio-id-trending-post',
  'cluster-affinity-postv2',
  'creator-affinity-post',
  'creator-affinity-post-backPress',
  'engaged-creator-affinity-post',
  'ffm-session-embedding',
  'ffm-vplay98-tf_recall_24hr_interaction',
  'ffm-weighted-play-realtimecosine-[1 1 1 1 1 1]',
  'ffm-weighted-vplay98-tf_recall_multilingual_durationsampling_24hr_60d-[1 1]',
  'interest-creator-affinity-post',
  'latestMinView',
  'lens-affinity-post',
  'promoted',
  'promoted-live-feed',
  'realTimeLabelD_overall_hof {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0}',
  'realTimeLabelD_overall_top {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0}',
  'sharecone-weighted-interaction_2kth_3day-[1]',
  'sharecon

In [89]:
def calc_stats(dt):
    dt = copy.deepcopy(dt)
    dt.groupby(["cg_rank", "pos"]).agg(cnt=pd.NamedAgg(column="cnt", aggfunc=sum)).reset_index()
    dt["cumsum"] = dt.groupby("cg_rank")["cnt"].cumsum()
    dt["cumsum_ratio"] = dt["cumsum"]/(1 + dt["pos"])
    dt = dt.groupby("cg_rank").agg(
        max_pos=pd.NamedAgg(column="pos", aggfunc=max),
        total_sum=pd.NamedAgg(column="cnt", aggfunc=sum),
        max_cumsum_ratio=pd.NamedAgg(column="cumsum_ratio", aggfunc=max),
        last_cumsum_ratio=pd.NamedAgg(column="cumsum_ratio", aggfunc=lambda ser: ser.values[-1])
    )
    dt.sort_values(by='last_cumsum_ratio', inplace = True, ascending=False)
    # normalize values
    dt["normalized_cumsum_ratio"] = dt["last_cumsum_ratio"] / dt["total_sum"].sum()
    return dt

In [90]:
calc_stats(df)

,max_pos,total_sum,max_cumsum_ratio,last_cumsum_ratio,normalized_cumsum_ratio
cg_rank,,,,,
sharecone-weighted-minview4-[1],9,401117828,3.999397e+08,5.013973e+07,6.228887e-03
sharecone-weighted-lt2k-[1 1 1 1 1],99,698149810,6.981411e+08,4.986784e+07,6.195111e-03
sharecone-weighted-realtimecosine_150k-[1 1 1 1 1 1],499,1503369378,1.478759e+09,3.758423e+07,4.669111e-03
sharecone-weighted-interaction_2kth_3day-[1],499,2154492810,2.138829e+09,3.122453e+07,3.879042e-03
promoted-live-feed,0,30847993,3.084799e+07,3.084799e+07,3.832264e-03
realTimeLabelD_overall_top {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0},29,81250925,8.125084e+07,1.015637e+07,1.261731e-03
sharecone-weighted-multilingual_2kth_3d_plc-[1 1],499,1856287106,1.845263e+09,9.926669e+06,1.233196e-03
realTimeLabelD_overall_hof {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0},29,87836864,8.588807e+07,8.783686e+06,1.091202e-03
sharecone-weightedlag-minview4-[1],9,38829314,3.882035e+07,7.765863e+06,9.647576e-04


In [91]:
dt = df[df["vplay98"]]
calc_stats(dt)

,max_pos,total_sum,max_cumsum_ratio,last_cumsum_ratio,normalized_cumsum_ratio
cg_rank,,,,,
sharecone-weighted-multilingual_2kth_3d_plc-[1 1],499,939719528,8.820465e+08,3.758878e+07,1.068493e-02
sharecone-weighted-realtimecosine_150k-[1 1 1 1 1 1],499,693567687,6.805500e+08,1.733919e+07,4.928814e-03
sharecone-weighted-interaction_2kth_3day-[1],499,1091102966,1.083963e+09,1.581309e+07,4.495005e-03
sharecone-weighted-lt2k-[1 1 1 1 1],99,170718602,1.705956e+08,1.219419e+07,3.466302e-03
sharecone-weighted-minview4-[1],9,47766165,4.734972e+07,6.823738e+06,1.939706e-03
audio-id-affinity-top-post,253,26336694,2.566758e+07,5.267339e+06,1.497286e-03
realTimeLabelD_overall_top {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0},29,28040048,2.803997e+07,3.505006e+06,9.963279e-04
realTimeLabelD_overall_hof {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0},29,26905111,2.641305e+07,2.690511e+06,7.648008e-04
promoted-live-feed,0,2313797,2.313797e+06,2.313797e+06,6.577166e-04


In [64]:
dt = df[df["is_shared"]]
calc_stats(dt)

,max_pos,max_cumsum_ratio,last_cumsum_ratio
cg_rank,,,
sharecone-weighted-realtimecosine_150k-[1 1 1 1 1 1],499,1.842561e+06,47454.950000
engaged-creator-affinity-post,504,1.503180e+05,30990.200000
sharecone-weighted-interaction_2kth_3day-[1],499,2.781209e+06,18275.516340
realTimeLabelD_overall_top {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0},29,1.390210e+05,17393.125000
sharecone-weighted-multilingual_2kth_3d_plc-[1 1],499,2.223015e+06,12122.497326
realTimeLabelD_overall_hof {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0},29,1.172760e+05,11896.200000
sharecone-weighted-lt2k-[1 1 1 1 1],99,2.731550e+05,9755.714286
sharecone-weighted-minview4-[1],9,6.589800e+04,8240.750000
ffm-session-embedding,99,4.774110e+05,5694.188889


In [75]:
dt2 = df.groupby(["cg_rank", "pos"]).agg(cnt=pd.NamedAgg(column="cnt", aggfunc=sum)).reset_index()
dt2 = dt2[dt2["cg_rank"]=="sharecone-weighted-realtimecosine_150k-[1 1 1 1 1 1]"]
pd.set_option("display.max_rows", 500)
print(dt2)

                                                cg_rank  pos       cnt
5420  sharecone-weighted-realtimecosine_150k-[1 1 1 ...    0  27963301
5421  sharecone-weighted-realtimecosine_150k-[1 1 1 ...    1  21922692
5422  sharecone-weighted-realtimecosine_150k-[1 1 1 ...    2  18759940
5423  sharecone-weighted-realtimecosine_150k-[1 1 1 ...    3  16696443
5424  sharecone-weighted-realtimecosine_150k-[1 1 1 ...    4  15197322
5425  sharecone-weighted-realtimecosine_150k-[1 1 1 ...    5  14039948
5426  sharecone-weighted-realtimecosine_150k-[1 1 1 ...    6  13124341
5427  sharecone-weighted-realtimecosine_150k-[1 1 1 ...    7  12354723
5428  sharecone-weighted-realtimecosine_150k-[1 1 1 ...    8  11703958
5429  sharecone-weighted-realtimecosine_150k-[1 1 1 ...    9  11144660
5430  sharecone-weighted-realtimecosine_150k-[1 1 1 ...   10  10658200
5431  sharecone-weighted-realtimecosine_150k-[1 1 1 ...   11  10224616
5432  sharecone-weighted-realtimecosine_150k-[1 1 1 ...   12   9843418
5433  

In [100]:
cands = pd.read_csv("cands_big.tsv", header=None, names=["cnt", "pos", "cg_rank"], sep="\t")
cands.groupby("cg_rank").agg(
    cnt = pd.NamedAgg(column="cnt", aggfunc=sum),
    max_pos = pd.NamedAgg(column="pos", aggfunc=max)
).sort_values(by='cnt', ascending=False)

,cnt,max_pos
cg_rank,,
sharecone-weighted-interaction_2kth_3day-[1],1256628892,499
sharecone-weighted-multilingual_2kth_3d_plc-[1 1],1177741845,499
sharecone-weighted-realtimecosine_150k-[1 1 1 1 1 1],1177520925,499
similar-creator-affinity-post,299118358,3275
creator-affinity-post,288942825,3303
ffm-session-embedding,272841448,499
similar-engaged-creator-affinity-post,253368464,5492
interest-creator-affinity-post,193656259,2237
engaged-creator-affinity-post,173218636,5500


In [101]:
cands = pd.read_csv("cands_1day.tsv", header=None, names=["cnt", "pos", "cg_rank"], sep="\t")
cands.groupby("cg_rank").agg(
    cnt = pd.NamedAgg(column="cnt", aggfunc=sum),
    max_pos = pd.NamedAgg(column="pos", aggfunc=max)
).sort_values(by='cnt', ascending=False)

,cnt,max_pos
cg_rank,,
sharecone-weighted-realtimecosine_150k-[1 1 1 1 1 1],4634581,499
sharecone-weighted-interaction_2kth_3day-[1],4253729,499
sharecone-weighted-multilingual_2kth_3d_plc-[1 1],4119667,499
ffm-session-embedding,2445870,99
similar-engaged-creator-affinity-post,2236921,524
similar-creator-affinity-post,1404630,524
audio-id-trending-post,1316645,99
sharecone-weighted-lt2k-[1 1 1 1 1],1139681,99
engaged-creator-affinity-post,1102545,521
